In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import json

when you first time run the script, you need to create a shortcut of the data folder (under ICT-Gender Bias shared folder) to your google drive

In [ ]:
os.chdir('/content/drive/MyDrive/data')

In [ ]:
indeed = []
for i, line in enumerate(open('indeed.json', 'r', encoding ='utf-8')):
    if i == 50000:
      break
    newline = json.loads(line)
    indeed.append(newline)


In [ ]:
len(indeed)

50000

In [ ]:
job_ads = [job.get('sentence') for job in indeed]

In [ ]:
import pandas as pd
# wordlist = pd.read_csv('wordlists.csv')
# wordlist_eng = wordlist[wordlist['lang']=='en']

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 55.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 21.6 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn.functional as F
from transformers import BertTokenizer, BertModel

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertModel.from_pretrained("bert-base-uncased", output_hidden_states = True)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

### Sentence Transformer & Trigrams

In [ ]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.1 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=875aa7c6e9657d092359b9b89cfe341c51a4d17a87601856e17451d16e26a018
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence_transformers


In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
import string
remove_punctuation = dict((ord(char), None) for char in string.punctuation)
remove_punctuation[8220] = None
remove_punctuation[8221] = None
remove_punctuation[8217] = None

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# traits
traits_1 = ['self-reliance', 'capable', 'challenging', 'independent', 'persistent', 'proactive']

traits_dict = {
    'rational' : ['rational', 'logical', 'analytical'], 
    'independent' : ['independent', 'self-reliant', 'autonomous'], 
    'leader': ['leader', 'supervisor', 'dominant'], 
    'risk taker': ['courageous', 'adventurous, bold'],
    'competitive' : ['competitive', 'strive', 'challenge'],
    'decisive' : ['decisive', 'resolute', 'persistent'],
    'ambitious' : ['ambitious', 'determined', 'driven'],
    'proactive' : ['proactive', 'assertive', 'confident'],
    'expert' : ['expert', 'skilled', 'adept'],
    'sensitive' : ['perceptive', 'intuitive', 'understanding'],
    'cooperative': ['cooperative', 'social', 'teamplayer'],
    'supportive' : ['supportive', 'assisting', 'aide'],
    'lawful' : ['lawful', 'grounded', 'careful'],
    'empathetic' : ['compassionate', 'empathetic', 'kind'],
    'agreeable' : ['compromising', 'accomodating', 'agreeable'],
    'easy-going' : ['reasonable', 'comfortable', 'calm'],
    'flexible' : ['flexible', 'patient', 'tolerant'],
    'learning' : ['learning', 'curious', 'respectful']
    }

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')

def cos_simi(job_ad, trait):

  score_dict = dict()
  trait_embd = model.encode(' '.join(trait))
  trait_embd = torch.Tensor(trait_embd)

  for sent in job_ad:
    sent = sent.translate(remove_punctuation)
    tokens = nltk.word_tokenize(sent)
    tokens = [i.lower() for i in tokens]
    tokens = [i for i in tokens if i not in stopwords.words('english')]
    if len(tokens) > 2:
      trigrams = [tokens[i:i+3] for i in range(len(tokens)-2)]
      # trigrams = nltk.trigrams(tokens)
      for triples in list(trigrams):
        triple_text = ' '.join(triples)
        tri_embd = model.encode(triple_text)
        tri_embd = torch.Tensor(tri_embd)
        cos_sim = F.cosine_similarity(tri_embd, trait_embd, dim=0)
        score_dict[triple_text] = cos_sim.item()
  
  return score_dict

In [ ]:
scores = {}
for trait in ['competitive', 'cooperative']: 
  scores[trait] = pd.Series(cos_simi(job_ads[49821], traits_dict[trait]))



In [ ]:
pd.DataFrame(scores)

,competitive,cooperative
siamo recruitment excellent,0.304560,0.218302
recruitment excellent opportunity,0.271272,0.105166
excellent opportunity 1st,0.192453,0.093176
opportunity 1st line,0.202795,0.044810
1st line support,0.114330,0.145314
...,...,...
job types fulltime,0.090900,0.012678
types fulltime permanent,0.102781,0.050488
salary £1900000 £2400000,-0.016176,-0.030428
£1900000 £2400000 per,0.003487,0.009055


In [ ]:
for trait1 in traits_dict:

  for trait2 in traits_dict:
    print(f"{trait1} \t {trait2} \t {cos_simi([' '.join(traits_dict[trait1])], traits_dict[trait2])}")

rational 	 rational 	 [('rational logical analytical', 1.0000001192092896)]
rational 	 independent 	 [('rational logical analytical', 0.05676702782511711)]
rational 	 leader 	 [('rational logical analytical', 0.07552224397659302)]
rational 	 risk taker 	 [('rational logical analytical', 0.12309960275888443)]
rational 	 competitive 	 [('rational logical analytical', 0.13682307302951813)]
rational 	 decisive 	 [('rational logical analytical', 0.19218523800373077)]
rational 	 ambitious 	 [('rational logical analytical', 0.19602175056934357)]
rational 	 proactive 	 [('rational logical analytical', 0.20841734111309052)]
rational 	 expert 	 [('rational logical analytical', 0.1366332620382309)]
rational 	 sensitive 	 [('rational logical analytical', 0.18296970427036285)]
rational 	 cooperative 	 [('rational logical analytical', 0.08009475469589233)]
rational 	 supportive 	 [('rational logical analytical', 0.06188463047146797)]
rational 	 lawful 	 [('rational logical analytical', 0.15696668624

In [ ]:
cos_simi([' '.join(traits_dict[trait1])], traits_dict['rational'])

[('learning curious respectful', 0.18174485862255096)]